In [80]:
import glob
import os
import scipy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline


def load_sound_files(dirs):
    raw_sounds = []
    labels = []
    for directory in dirs:
        music_label = (directory.split("/"))[6]
        audio_files = glob.glob(directory + '/*[0-9].wav')
        for file in audio_files:
            sr,X = scipy.io.wavfile.read(file)
            fft_feature = abs(scipy.fft(X[:10000]))
            #print(fft_feature.shape)
            #print("sr:{0} length: {1}".format(sr, len(fft_feature)))
            raw_sounds.append(fft_feature)
            labels.append(music_label)
    return raw_sounds, labels

def linear_classifier(raw_sounds, T):
    T_kesslerized = []

    for i in range(len((T))):
        if T[i] == 'pop':
            T_kesslerized.append(np.array([ 1, -1, -1, -1, -1, -1, -1, -1, -1, -1]))
        if T[i] == 'metal':
            T_kesslerized.append(np.array([ -1, 1, -1, -1, -1, -1, -1, -1, -1, -1]))
        if T[i] == 'disco':
            T_kesslerized.append(np.array([ -1, -1, 1, -1, -1, -1, -1, -1, -1, -1]))
        if T[i] == 'blues':
            T_kesslerized.append(np.array([ -1, -1, -1, 1, -1, -1, -1, -1, -1, -1]))
        if T[i] == 'reggae':
            T_kesslerized.append(np.array([ -1, -1, -1, -1, 1, -1, -1, -1, -1, -1]))
        if T[i] == 'classical':
            T_kesslerized.append(np.array([ -1, -1, -1, -1, -1, 1, -1, -1, -1, -1]))
        if T[i] == 'rock':
            T_kesslerized.append(np.array([ -1, -1, -1, -1, -1, -1, 1, -1, -1, -1]))
        if T[i] == 'hiphop':
            T_kesslerized.append(np.array([ -1, -1, -1, -1, -1, -1, -1, 1, -1, -1]))
        if T[i] == 'country':
            T_kesslerized.append(np.array([ -1, -1, -1, -1, -1, -1, -1, -1, 1, -1]))
        if T[i] == 'jazz':
            T_kesslerized.append(np.array([ -1, -1, -1, -1, -1, -1, -1, -1, -1, 1]))
    
    Xa = np.insert(raw_sounds, 0, 1, axis=1)
    Xainv = np.linalg.pinv(Xa)
    w = np.dot(Xainv, T_kesslerized)
    return w
    

def unkesslerize(types):
    for i in range(len(types)):
        types[i][np.argmax(types[i])] = 1
        types[i][types[i] < 1] = -1

    types_unkesslerized= []
    for i in range(len(types)):
        if np.array_equal(types[i],np.array([ 1, -1, -1, -1, -1, -1, -1, -1, -1, -1])):
            types_unkesslerized.append('pop')
        elif np.array_equal(types[i],np.array([ -1, 1, -1, -1, -1, -1, -1, -1, -1, -1])):
            types_unkesslerized.append('metal')
        elif np.array_equal(types[i],np.array([ -1, -1, 1, -1, -1, -1, -1, -1, -1, -1])):
            types_unkesslerized.append('disco')
        elif np.array_equal(types[i],np.array([ -1, -1, -1, 1, -1, -1, -1, -1, -1, -1])):
            types_unkesslerized.append('blues')
        elif np.array_equal(types[i],np.array([ -1, -1, -1, -1, 1, -1, -1, -1, -1, -1])):
            types_unkesslerized.append('reggae')
        elif np.array_equal(types[i],np.array([ -1, -1, -1, -1, -1, 1, -1, -1, -1, -1])):
            types_unkesslerized.append('classical')
        elif np.array_equal(types[i],np.array([ -1, -1, -1, -1, -1, -1, 1, -1, -1, -1])):
            types_unkesslerized.append('rock')
        elif np.array_equal(types[i],np.array([ -1, -1, -1, -1, -1, -1, -1, 1, -1, -1])):
            types_unkesslerized.append('hiphop')
        elif np.array_equal(types[i],np.array([ -1, -1, -1, -1, -1, -1, -1, -1, 1, -1])):
            types_unkesslerized.append('country')
        elif np.array_equal(types[i],np.array([ -1, -1, -1, -1, -1, -1, -1, -1, -1, 1])):
            types_unkesslerized.append('jazz')
        else:
            print(types[i])

    return types_unkesslerized

if __name__ == '__main__':
    path = "/Users/sharath/ML/musicanalysis/genres/*"
    dirs = glob.glob(path)
    raw_sounds, labels = load_sound_files(dirs)
    raw_sounds = np.vstack((raw_sounds))
    labels = np.array(labels)
    #print(raw_sounds.shape, labels.shape)
    print("training")
    w = linear_classifier(raw_sounds, labels)
    
    path_pop= ["/Users/sharath/ML/musicanalysis/genres/pop"]
    queries, labels = load_sound_files(path_pop)
    print("test - pop")
    queries = np.array(queries)
    queriesa = np.insert(queries, 0, 1, axis = 1)
    types = np.dot(queriesa, w)
    print(types.shape)
    print("done")
    types_unkesslerized = np.array(unkesslerize(types))
    print(types_unkesslerized.shape)
    

(1000, 10000) (1000,)
test
(100, 10)
done
(100,)


(6600, 15)
(6600,)
(6600,)
